In [ ]:
import os, time
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
device = "cuda"

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class convNet(nn.Module):
    def __init__(self,in_channels):
        super(convNet,self).__init__()
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=64,kernel_size=3)
        self.bn1 = nn.BatchNorm2d(num_features=64,momentum=0.1)
        self.c2 = nn.Conv2d(64,64,3)
        self.bn2 = nn.BatchNorm2d(num_features=64,momentum=0.1)
        self.d1 = nn.Dropout(0.2)
        
        self.c3 = nn.Conv2d(64,128,3)
        self.bn3 = nn.BatchNorm2d(128,0.1)
        self.c4 = nn.Conv2d(128,128,3)
        self.bn4 = nn.BatchNorm2d(128,0.1)
        self.d2 = nn.Dropout(0.2)
        
        self.c5 = nn.Conv2d(128,256,3)
        self.bn5 = nn.BatchNorm2d(256,0.1)
        self.c6 = nn.Conv2d(256,256,3)
        self.bn6 = nn.BatchNorm2d(256,0.1)
        self.d3 = nn.Dropout(0.2)
        
        self.fc1 = nn.Linear(256*2*2,256)
        self.fc2 = nn.Linear(256,256)
        self.out = nn.Linear(256,10)

    def forward(self,x):
        x = F.leaky_relu(self.bn1(self.c1(x)),negative_slope=0.1)
        x = F.leaky_relu(self.bn2(self.c2(x)),0.1)
        x = F.max_pool2d(x,2)
        x = self.d1(x)
        
        x = F.leaky_relu(self.bn3(self.c3(x)),0.1)
        x = F.leaky_relu(self.bn4(self.c4(x)),0.1)
        x = F.max_pool2d(x,2)
        x = self.d2(x)
        
        x = F.leaky_relu(self.bn5(self.c5(x)),0.1)
        x = F.leaky_relu(self.bn6(self.c6(x)),0.1)
        
        x = F.max_pool2d(x,2)        
        x = self.d3(x)
        
        x = x.view(-1, 256*2*2) #reshape
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)
                

In [ ]:
count_parameters(convNet(1))

In [ ]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', 'x'.join(str(x) for x in list(param.size())), '=', num_param)
            else:
                print(name, ':', num_param)
            total_param += num_param
    return total_param

In [ ]:
trans = transforms.Compose([
        transforms.RandomResizedCrop(28),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(180),
        transforms.ToTensor(),  #Take Image as input and convert to tensor with value from 0 to1
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225])
    ])

trans_val = transforms.ToTensor()

class KMnistDataset(Dataset):
    def __init__(self,data_len=None, is_validate=False,validate_rate=None):
        self.is_validate = is_validate
        self.data = pd.read_csv("./dataset/train.csv")
        print("data shape:", np.shape(self.data))
        if data_len == None:
            data_len = len(self.data)
        
        if self.is_validate:
            self.len = int(data_len*validate_rate)
            self.offset = int(data_len*(1-validate_rate))
            self.transform = trans_val
        else:
            self.len = int(data_len*(1-validate_rate))
            self.offset = 0
            self.transform = trans
        
    def __getitem__(self, idx):
        idx += self.offset
#         if self.offset !=0:
#             print(idx)
        img = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
        label = self.data.iloc[idx, 0]  #(num,)
        img = Image.fromarray(img)

        img = self.transform(img)     #value: 0~1, shape:(1,28,28)
        label = torch.as_tensor(label, dtype=torch.uint8)    #value: 0~9, shape(1)
            
        return img, label

    def __len__(self):
        return self.len

In [ ]:
batch_size = 32
num_workers = 8
# kmnist_dataset = datasets.ImageFolder(kmnist_dataset, transform=None)

vr = 0.2
train_dataset = KMnistDataset(data_len=None,is_validate=False, validate_rate=vr)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

validate_dataset = KMnistDataset(data_len=None,is_validate=True, validate_rate=vr)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
kmnist_dataset.data.head(100)

In [ ]:
type(kmnist_dataset.data.iloc[20,1:].values)  #numpy ndarray
type(kmnist_dataset.data.iloc[20,0])  #numpy int64
kmnist_dataset.data.head(5)

In [ ]:
def main():
    epochs = 500000
    max_acc = 0
    data_num = 0
    loss_avg = 0
    lr = 1e-4
    validate_ep = 100
    
    model = convNet(in_channels=1)
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=lr)
#     optimizer = torch.optim.SGD(net_Momentum.parameters(),lr=Learning_rate,momentum=0.8,nesterov=True)
#     optimizer = torch.optim.RMSprop(net_RMSprop.parameters(),lr=Learning_rate,alpha=0.9)
#     optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=(0.9,0.99))
#     optimizer = torch.optim.Adagrad(net_Adagrad.parameters(),lr=Learning_rate)
    
    for ep in range(0,epochs):
        model.train()
        for idx, data in enumerate(train_loader):
            img, target = data
            img, target = img.to(device), target.to(device,dtype=torch.long)
#             print(np.shape(img),np.shape(target)) #Tensor(4,1,28,28), Tensor(4)
#             print(np.max(img.cpu().numpy()),np.min(img.cpu().numpy())) #1.0 0.0
            pred = model(img)
            print(pred.size())   #(32,10)
            print(target.size()) #(32,)
            ###Input shape must be pred:, target:
            loss = criterion(pred,target)
        
            loss_avg += loss.item()
            data_num += img.size(0)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        ###Evaluate Train Loss 
        if ep%10 == 0:
            loss_avg /= data_num
            print("Ep:{}, loss:{}".format(ep,loss_avg))
            loss_avg = 0
            data_num = 0
        
        ###Validation
        if ep!=0 and ep%validate_ep == 0:
            model.eval()
            acc = 0
            data_num  = 0
            with torch.no_grad():
                for idx, data in enumerate(validate_loader):
                    img, target = data
                    img, target = img.to(device), target.to(device,dtype=torch.long)
                    pred = model(img)

                    # print(pred)
                    _,pred_class = torch.max(pred.data, 1)
#                     print(pred_class)
                    
                    data_num += img.size(0)
                    acc += (pred_class == target).sum().item()
#                     print("pred_class:",pred_class)
#                     print("target:",target)
#                     print("tmp:",tmp)

#             print("data_num:", data_num)
            acc /= data_num
            if acc > max_acc:
                max_acc = acc
                print("===================Best Model Saved:==================")
                print("Episode:{}, Accuracy:{:.4f}%".format(ep,max_acc*100))
                if max_acc > 0.7 :
                    torch.save(model.state_dict(), "./Kmnist_saved_model/ep{}_acc{:.4f}".format(ep,max_acc))
                print("======================================================")
            
if __name__ == "__main__":
    main()
    